# 3. Historical Trends

For each region, what kind of changes do we see over time?

In [2]:
# set up path to app credentials - see exploration/README.md
%env GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json

# set up bigquery client
from google.cloud import bigquery
bq = bigquery.Client()

env: GOOGLE_APPLICATION_CREDENTIALS=../google_app_credentials.json


In [3]:
# set up some dependencies
import json, time, random, csv
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import descartes # for plotting with geopanadas

In [4]:
resp = bq.query('''
    SELECT DISTINCT do_date AS year
    FROM `eosc410-project.data.acs_cbsa_20*`
    ORDER BY do_date ASC
''')
years = [row["year"] for row in resp]

## Air Quality Over Time

In [28]:
def collect_measurements(year: str, epa: pd.DataFrame):
    regions = epa['epa_site']
    regions_query = ','.join(['"%s"' % r for r in regions])
    # TODO: we need to thin down the collected measurements significantly
    resp = bq.query('''
        WITH query AS ( SELECT [ %s ] AS sites )
        SELECT DISTINCT
            CONCAT(epa.state_code, ".", epa.county_code, ".", epa.site_num) AS epa_site,
            epa.parameter_code,
            epa.sample_duration,
            epa.arithmetic_mean,
            epa.arithmetic_standard_dev,
            epa.units_of_measure,
            epa.metric_used
        FROM
            query, `eosc410-project.data.epa_air_quality_annual` AS epa
        WHERE
            epa.year = %s
            AND CONCAT(epa.state_code, ".", epa.county_code, ".", epa.site_num) IN UNNEST(query.sites)
    ''' % (regions_query, year))
    return resp.to_dataframe()

In [29]:
# test on one year
y = '2008'
epa = pd.read_csv('../_data/epa_to_acs_cbsa/%s.csv' % y)
print(collect_measurements(y, epa))

epa_site  parameter_code sample_duration  arithmetic_mean  \
0      16.001.0010           88111         24 HOUR         0.049383   
1      16.001.0010           88388         24 HOUR         0.577274   
2      16.001.0010           88375         24 HOUR         0.528522   
3      16.001.0010           88165         24 HOUR         0.136442   
4      16.001.0010           88154         24 HOUR         0.000107   
...            ...             ...             ...              ...   
74045  51.840.0002           81102         24 HOUR        18.032787   
74046  24.510.0035           81102         24 HOUR        27.777778   
74047  22.121.0001           81102         24 HOUR        31.000000   
74048  22.121.0001           81102         24 HOUR        29.966667   
74049  11.001.0041           81102         24 HOUR        19.827586   

       arithmetic_standard_dev               units_of_measure      metric_used  
0                     0.046137    Micrograms/cubic meter (LC)  Observed Valu

In [6]:
# collect all records in a time series
timeseries = pd.DataFrame()
# for y in ['2007']:
    # epa = pd.read_csv('../_data/epa_to_acs_cbsa/%s.csv' % y)
    # TODO

18795
